In [4]:
import re
import json
import pdfplumber
from docx import Document
import pandas as pd

In [5]:
keywords = {
    "violence": ["ударил", "кровь", "убил", "стреляет", "пистолет", "драка", "нож", "труп"],
    "sexual": ["поцеловал", "разделась", "постель", "эрот", "секс", "интим"],
    "profanity": ["черт", "блин", "сука", "гад", "дерьмо", "падла", "еб", "хер"],
    "alcohol_drugs": ["водка", "пьяный", "алкоголь", "наркотик", "косяк", "виски", "курит"],
    "scary": ["кричит", "труп", "монстр", "страшно", "крик", "кровь", "тень"]
}

In [6]:
def read_file(path):
    if path.endswith(".pdf"):
        text = ""
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
        return text
    elif path.endswith(".docx"):
        doc = Document(path)
        return "\n".join([p.text for p in doc.paragraphs])
    else:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()

In [7]:
def split_scenes(text):
    parts = re.split(r'(СЦЕНА\s*\d*\.|INT\.|EXT\.|ИНТ\.|ЭКСТ\.)', text, flags=re.IGNORECASE)
    scenes = []
    current = ""
    for part in parts:
        if re.match(r'(СЦЕНА|INT|EXT|ИНТ|ЭКСТ)', part, flags=re.IGNORECASE):
            if current.strip():
                scenes.append(current.strip())
            current = part
        else:
            current += " " + part
    if current.strip():
        scenes.append(current.strip())
    return [s for s in scenes if len(s.split()) > 3]

# ==== анализ одной сцены ====
def analyze_scene(scene):
    result = {}
    scene_lower = scene.lower()
    for category, words in keywords.items():
        count = sum(w in scene_lower for w in words)
        if count == 0:
            severity = "None"
        elif count == 1:
            severity = "Mild"
        elif 2 <= count <= 3:
            severity = "Moderate"
        else:
            severity = "Severe"
        result[category] = {"count": count, "severity": severity}
    return result

# ==== агрегирование итогового рейтинга ====
def get_age_rating(scene_results):
    severity_map = {"None": 0, "Mild": 1, "Moderate": 2, "Severe": 3}
    max_score = 0
    for scene in scene_results:
        for cat, data in scene.items():
            score = severity_map[data["severity"]]
            if score > max_score:
                max_score = score
    if max_score == 0:
        return "0+"
    elif max_score == 1:
        return "12+"
    elif max_score == 2:
        return "16+"
    else:
        return "18+"

# ==== запуск анализа ====
def analyze_script(path):
    text = read_file(path)
    scenes = split_scenes(text)
    results = []
    for scene in scenes:
        res = analyze_scene(scene)
        results.append(res)
    age_rating = get_age_rating(results)

    # сбор отчета
    report = []
    for i, scene in enumerate(scenes):
        entry = {"scene_id": i + 1, "text": scene}
        entry.update({k: v["severity"] for k, v in results[i].items()})
        report.append(entry)

    df = pd.DataFrame(report)
    df.to_csv("scene_analysis.csv", index=False)
    with open("report.json", "w", encoding="utf-8") as f:
        json.dump({"age_rating": age_rating, "scenes": report}, f, ensure_ascii=False, indent=2)

    print(f"\n✅ Итоговый возрастной рейтинг: {age_rating}")
    print(f"📊 Сохранены файлы: report.json, scene_analysis.csv")

# ==== запуск ====
if __name__ == "__main__":
    path = input("Введите путь к сценарию (.txt/.pdf/.docx): ").strip()
    analyze_script(path)


✅ Итоговый возрастной рейтинг: 16+
📊 Сохранены файлы: report.json, scene_analysis.csv
